# 🛡️ Resilience Testing - Classification

<div style="background-color: #e3f2fd; padding: 15px; border-radius: 5px; border-left: 5px solid #2196F3;">
<b>📓 Information</b><br>
<b>Level:</b> Intermediate/Advanced<br>
<b>Time:</b> 20 minutes<br>
<b>Dataset:</b> Breast Cancer (sklearn)<br>
<b>Prerequisite:</b> Basic understanding of ML models
</div>

## 🎯 Objectives
- ✅ Resilience testing for **classification** problems
- ✅ Generate **interactive HTML report**
- ✅ Export results to **JSON format**
- ✅ Analyze **data drift and concept drift**
- ✅ Evaluate **model stability** under distribution shifts

## 📚 Why Resilience Testing?

### Critical Contexts

#### 🏥 Medicine - Disease Diagnosis
- **Problem**: Model trained on Hospital A data, deployed in Hospital B
- **Challenge**: Different patient demographics, equipment, protocols
- **Impact**: Model performance may degrade silently
- **Solution**: Resilience testing detects distribution shifts early

#### 💳 Finance - Credit Scoring
- **Problem**: Model trained on pre-pandemic data
- **Challenge**: Economic conditions changed dramatically
- **Impact**: Credit risk patterns shifted significantly
- **Solution**: Monitor resilience to detect when retraining is needed

#### 🔒 Security - Fraud Detection
- **Problem**: Fraudsters constantly evolve their tactics
- **Challenge**: New fraud patterns emerge continuously
- **Impact**: Model becomes less effective over time
- **Solution**: Resilience metrics trigger retraining workflows

### Types of Drift

1. **Data Drift** (Covariate Shift)
   - Feature distributions change
   - Example: Age distribution shifts from 20-40 to 40-60

2. **Concept Drift**
   - Relationship between features and target changes
   - Example: What constitutes "fraud" evolves over time

3. **Performance Degradation**
   - Model accuracy decreases on new data
   - May be caused by data or concept drift

## 1️⃣ Setup - Binary Classification Problem

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report
from deepbridge import DBDataset, Experiment
import os

# Configure visualizations
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Load Breast Cancer dataset
cancer = load_breast_cancer()
df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
df['target'] = cancer.target  # 0 = malignant, 1 = benign

print(f"📊 Dataset: {df.shape}")
print(f"🏥 Target: Cancer diagnosis (0=malignant, 1=benign)")
print(f"\n📈 Class distribution:")
print(df['target'].value_counts())
print(f"\n📊 Class balance:")
print(df['target'].value_counts(normalize=True))

2025-11-12 22:40:24,481 - deepbridge.reports - DEBUG - Using refactored FairnessDataTransformer
2025-11-12 22:40:24,488 - deepbridge.reports - INFO - Successfully imported radar chart fix
2025-11-12 22:40:24,489 - deepbridge.reports - INFO - Successfully patched EnhancedUncertaintyCharts.generate_model_metrics_comparison
2025-11-12 22:40:24,489 - deepbridge.reports - INFO - Successfully applied enhanced_charts patch
2025-11-12 22:40:24,491 - deepbridge.reports - INFO - Successfully loaded UncertaintyChartGenerator
2025-11-12 22:40:24,493 - deepbridge.reports - INFO - Successfully imported and initialized SeabornChartGenerator
2025-11-12 22:40:24,494 - deepbridge.reports - INFO - SeabornChartGenerator has_visualization_libs: True
2025-11-12 22:40:24,495 - deepbridge.reports - INFO - Available chart methods: ['bar_chart', 'boxplot_chart', 'coverage_analysis_chart', 'detailed_boxplot_chart', 'distribution_grid_chart', 'feature_comparison_chart', 'feature_importance_chart', 'feature_psi_ch

## 2️⃣ Train Classification Model

In [2]:
X = df.drop('target', axis=1)
y = df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Train RandomForest Classifier
model = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=10)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)
accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba[:, 1])

print(f"✅ Model trained!")
print(f"📊 Accuracy: {accuracy*100:.2f}%")
print(f"📊 ROC-AUC: {auc:.3f}")
print(f"\n📋 Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Malignant', 'Benign']))

✅ Model trained!
📊 Accuracy: 95.61%
📊 ROC-AUC: 0.994

📋 Classification Report:
              precision    recall  f1-score   support

   Malignant       0.95      0.93      0.94        42
      Benign       0.96      0.97      0.97        72

    accuracy                           0.96       114
   macro avg       0.96      0.95      0.95       114
weighted avg       0.96      0.96      0.96       114



## 3️⃣ Create Experiment

In [3]:
dataset = DBDataset(
    data=df,
    target_column='target',
    model=model,
    test_size=0.2,
    random_state=42,
    dataset_name='Breast Cancer Classification'
)

exp = Experiment(
    dataset=dataset,
    experiment_type='binary_classification',
    tests=['resilience'],  # Specify resilience test
    random_state=42
)

print("✅ Experiment created!")

✅ Initial model evaluation complete: RandomForestClassifier
✅ Experiment created!


## 4️⃣ Run Resilience Test

<div style="background-color: #fff3e0; padding: 15px; border-radius: 5px; border-left: 5px solid #ff9800;">
<b>ℹ️ Configuration:</b> Using 'full' config for comprehensive resilience analysis with multiple drift detection methods.
</div>

In [4]:
print("🧪 Running resilience test...\n")

# Use run_tests() to store results internally for save_html() and save_json()
experiment_result = exp.run_tests(config_name='full')

print("\n✅ Resilience test completed!")
print(f"\n📊 Result type: {type(experiment_result)}")

# Access the resilience result
resilience_result = experiment_result.get_result('resilience')
print(f"📊 Resilience result type: {type(resilience_result)}")

🧪 Running resilience test...

✅ Resilience Tests Finished!
🎉 Test completed successfully: resilience

✅ Resilience test completed!

📊 Result type: <class 'deepbridge.core.experiment.results.ExperimentResult'>
📊 Resilience result type: <class 'dict'>


## 5️⃣ Generate Interactive HTML Report

### Export comprehensive interactive report with:
- 📊 **Performance Analysis**: Model stability metrics
- 📈 **Drift Detection**: Data and concept drift analysis
- 🎯 **Feature Impact**: Features most affected by distribution shifts
- 📉 **Resilience Scores**: Overall model robustness metrics

In [5]:
# Create output directory
output_dir = './outputs/resilience_classification'
os.makedirs(output_dir, exist_ok=True)

# Generate interactive HTML report
html_output_path = os.path.join(output_dir, 'resilience_classification_interactive.html')

print("📝 Generating interactive HTML report...\n")

report_path = exp.save_html(
    test_type='resilience',
    file_path=html_output_path,
    model_name='RandomForest Classifier',
    report_type='interactive'
)

print(f"\n✅ Interactive report generated!")
print(f"📂 Location: {report_path}")
print(f"\n💡 Open the HTML file in your browser to explore:")
print(f"   - Performance Overview")
print(f"   - Drift Detection Analysis")
print(f"   - Feature Distribution Shifts")
print(f"   - Interactive Plotly charts")

📝 Generating interactive HTML report...

2025-11-12 22:40:26,973 - deepbridge.reports - INFO - Generating SIMPLE resilience report to: /home/guhaase/projetos/DeepBridge/examples/notebooks/03_validation_tests/outputs/resilience_classification/resilience_classification_interactive.html
2025-11-12 22:40:26,974 - deepbridge.reports - INFO - Report type: interactive
2025-11-12 22:40:26,974 - deepbridge.reports - INFO - ================================================================================
2025-11-12 22:40:26,975 - deepbridge.reports - INFO - TRANSFORMING RESILIENCE DATA FOR REPORT (SIMPLE)
2025-11-12 22:40:26,975 - deepbridge.reports - INFO - ================================================================================
2025-11-12 22:40:26,976 - deepbridge.reports - INFO - Top-level keys in results: ['primary_model', 'alternative_models', 'initial_results', 'initial_model_evaluation', 'config', 'experiment_type', 'model_type']
2025-11-12 22:40:26,977 - deepbridge.reports - INFO 

## 6️⃣ Export Results to JSON

### JSON export includes:
- 🔍 **Experiment Info**: Configuration and metadata
- 📊 **Test Results**: Complete resilience analysis data
- 🎯 **Model Evaluation**: Model metrics and performance
- 📈 **Drift Metrics**: Distribution shift measurements
- 🌟 **Feature Analysis**: Per-feature drift statistics

In [6]:
# Export to JSON (COMPACT MODE for AI validation)
json_output_path = os.path.join(output_dir, 'resilience_classification_results.json')
json_compact_path = os.path.join(output_dir, 'resilience_classification_results_COMPACT.json')

print("📝 Exporting results to JSON...\n")

# Full JSON (with all data)
json_path_full = exp._experiment_result.save_json(
    test_type='resilience',
    file_path=json_output_path,
    include_summary=True,
    compact=False  # Full data
)

# Compact JSON (optimized for AI validation)
json_path_compact = exp._experiment_result.save_json(
    test_type='resilience',
    file_path=json_compact_path,
    include_summary=True,
    compact=True  # Remove large arrays, keep only essentials
)

# Compare file sizes
import os
size_full = os.path.getsize(json_path_full) / 1024  # KB
size_compact = os.path.getsize(json_path_compact) / 1024  # KB
reduction = ((size_full - size_compact) / size_full) * 100

print(f"\n✅ JSON files exported successfully!")
print(f"\n📊 FILE SIZES:")
print(f"   Full JSON:    {size_full:>8.2f} KB")
print(f"   Compact JSON: {size_compact:>8.2f} KB")
print(f"   Reduction:    {reduction:>8.1f}%")

print(f"\n📂 LOCATIONS:")
print(f"   Full:    {json_path_full}")
print(f"   Compact: {json_path_compact}")

print(f"\n📋 COMPACT JSON STRUCTURE (optimized for AI):")
print(f"   └─ experiment_info/")
print(f"      ├─ test_type, experiment_type, generation_time, config")
print(f"   └─ test_results/")
print(f"      └─ primary_model/")
print(f"         ├─ metrics (all model metrics)")
print(f"         ├─ resilience_score")
print(f"         ├─ drift_detected")
print(f"         └─ drift_analysis/")
print(f"            ├─ data_drift_score")
print(f"            ├─ concept_drift_score")
print(f"            └─ per_feature_drift (top affected features)")
print(f"   └─ initial_model_evaluation/ (compact)")
print(f"   └─ summary/  (AI-friendly analysis)")
print(f"      ├─ key_findings")
print(f"      ├─ drift_summary")
print(f"      ├─ performance_stability")
print(f"      └─ recommendations")

print(f"\n💡 USE CASES:")
print(f"   • Full JSON: Deep dive analysis, debugging, research")
print(f"   • Compact JSON: AI validation, automated testing, CI/CD pipelines")

📝 Exporting results to JSON...


✅ JSON files exported successfully!

📊 FILE SIZES:
   Full JSON:      308.84 KB
   Compact JSON:   304.81 KB
   Reduction:         1.3%

📂 LOCATIONS:
   Full:    /home/guhaase/projetos/DeepBridge/examples/notebooks/03_validation_tests/outputs/resilience_classification/resilience_classification_results.json
   Compact: /home/guhaase/projetos/DeepBridge/examples/notebooks/03_validation_tests/outputs/resilience_classification/resilience_classification_results_COMPACT.json

📋 COMPACT JSON STRUCTURE (optimized for AI):
   └─ experiment_info/
      ├─ test_type, experiment_type, generation_time, config
   └─ test_results/
      └─ primary_model/
         ├─ metrics (all model metrics)
         ├─ resilience_score
         ├─ drift_detected
         └─ drift_analysis/
            ├─ data_drift_score
            ├─ concept_drift_score
            └─ per_feature_drift (top affected features)
   └─ initial_model_evaluation/ (compact)
   └─ summary/  (AI-friendly 

## 7️⃣ Load and Analyze JSON Results

Demonstrate how to load and use the exported JSON

In [7]:
import json

print("="*80)
print("📊 ANALYZING COMPACT JSON (optimized for AI validation)")
print("="*80)

# Load COMPACT JSON results
with open(json_compact_path, 'r', encoding='utf-8') as f:
    results_json = json.load(f)

# Experiment Info
exp_info = results_json['experiment_info']
print(f"\n🔬 EXPERIMENT INFO:")
print(f"   Test Type: {exp_info['test_type']}")
print(f"   Experiment Type: {exp_info['experiment_type']}")
print(f"   Generation Time: {exp_info['generation_time']}")

# Summary (AI-friendly)
if 'summary' in results_json:
    summary = results_json['summary']
    
    print(f"\n🎯 KEY FINDINGS:")
    for finding in summary.get('key_findings', []):
        print(f"   • {finding}")
    
    print(f"\n📈 DRIFT SUMMARY:")
    drift = summary.get('drift_summary', {})
    for key, value in drift.items():
        print(f"   {key}: {value}")
    
    print(f"\n🔍 PERFORMANCE STABILITY:")
    perf = summary.get('performance_stability', {})
    for key, value in perf.items():
        print(f"   {key}: {value}")
    
    print(f"\n💡 RECOMMENDATIONS:")
    for rec in summary.get('recommendations', []):
        print(f"   • {rec}")

# Test Results (compact)
test_results = results_json['test_results']
primary = test_results.get('primary_model', {})

print(f"\n🔬 TEST RESULTS (Compact):")
print(f"   Resilience Score: {primary.get('resilience_score', 'N/A')}")
print(f"   Drift Detected: {primary.get('drift_detected', 'N/A')}")

# Drift Analysis
if 'drift_analysis' in primary:
    drift_analysis = primary['drift_analysis']
    print(f"\n📊 DRIFT ANALYSIS:")
    print(f"   Data Drift Score: {drift_analysis.get('data_drift_score', 'N/A')}")
    print(f"   Concept Drift Score: {drift_analysis.get('concept_drift_score', 'N/A')}")
    
    if 'per_feature_drift' in drift_analysis:
        print(f"\n🌟 TOP FEATURES WITH HIGHEST DRIFT:")
        print(f"   {'Feature':<30} {'Drift Score'}")
        print(f"   {'-'*50}")
        for feat, score in list(drift_analysis['per_feature_drift'].items())[:10]:
            print(f"   {feat:<30} {score:.4f}")

print(f"\n{'='*80}")
print(f"💾 COMPACT JSON IS OPTIMIZED FOR:")
print(f"{'='*80}")
print(f"✅ AI/LLM validation (smaller token count)")
print(f"✅ Automated testing pipelines")
print(f"✅ CI/CD integration")
print(f"✅ Quick drift detection")
print(f"✅ Summary-based decision making")

📊 ANALYZING COMPACT JSON (optimized for AI validation)

🔬 EXPERIMENT INFO:
   Test Type: resilience
   Experiment Type: binary_classification
   Generation Time: 2025-11-12 22:40:26

🔬 TEST RESULTS (Compact):
   Resilience Score: 1.0
   Drift Detected: N/A

💾 COMPACT JSON IS OPTIMIZED FOR:
✅ AI/LLM validation (smaller token count)
✅ Automated testing pipelines
✅ CI/CD integration
✅ Quick drift detection
✅ Summary-based decision making


## 8️⃣ Resilience Analysis Summary

Extract key insights from resilience testing

In [8]:
print("\n📊 RESILIENCE TESTING SUMMARY\n" + "="*70)

# Access resilience metrics
if 'resilience_score' in primary:
    resilience_score = primary['resilience_score']
    print(f"\n📈 OVERALL RESILIENCE:")
    print(f"   Score: {resilience_score:.3f}")
    
    # Quality assessment
    print(f"\n🎯 QUALITY ASSESSMENT:")
    
    if resilience_score >= 0.90:
        print(f"   ✅ Resilience: EXCELLENT (score ≥ 0.90)")
        print(f"      Model is highly stable under distribution shifts")
    elif resilience_score >= 0.75:
        print(f"   🟡 Resilience: GOOD (score ≥ 0.75)")
        print(f"      Model shows acceptable stability")
    elif resilience_score >= 0.60:
        print(f"   ⚠️  Resilience: MODERATE (score ≥ 0.60)")
        print(f"      Monitor model performance closely")
    else:
        print(f"   🔴 Resilience: LOW (score < 0.60)")
        print(f"      Consider retraining or model improvement")

# Drift detection
if 'drift_detected' in primary:
    drift_detected = primary['drift_detected']
    print(f"\n🔍 DRIFT DETECTION:")
    
    if drift_detected:
        print(f"   ⚠️  Drift DETECTED")
        print(f"      Data distribution has shifted significantly")
        print(f"      Action: Review drift analysis and consider retraining")
    else:
        print(f"   ✅ No significant drift detected")
        print(f"      Model remains valid for current data distribution")

# Drift analysis details
if 'drift_analysis' in primary:
    drift_analysis = primary['drift_analysis']
    
    print(f"\n📊 DRIFT ANALYSIS:")
    
    data_drift = drift_analysis.get('data_drift_score', 0)
    concept_drift = drift_analysis.get('concept_drift_score', 0)
    
    print(f"   Data Drift Score: {data_drift:.3f}")
    if data_drift > 0.3:
        print(f"      ⚠️  High data drift - feature distributions changed")
    elif data_drift > 0.1:
        print(f"      🟡 Moderate data drift detected")
    else:
        print(f"      ✅ Low data drift")
    
    print(f"\n   Concept Drift Score: {concept_drift:.3f}")
    if concept_drift > 0.3:
        print(f"      ⚠️  High concept drift - target relationships changed")
    elif concept_drift > 0.1:
        print(f"      🟡 Moderate concept drift detected")
    else:
        print(f"      ✅ Low concept drift")

# Recommendations
print(f"\n💡 RECOMMENDATIONS:")

if primary.get('drift_detected', False):
    print(f"   • URGENT: Investigate drift sources")
    print(f"   • Consider retraining with recent data")
    print(f"   • Implement drift monitoring in production")

if primary.get('resilience_score', 1.0) < 0.75:
    print(f"   • Model shows sensitivity to distribution shifts")
    print(f"   • Collect more diverse training data")
    print(f"   • Use ensemble methods to improve stability")
    print(f"   • Apply data augmentation techniques")

if primary.get('resilience_score', 1.0) >= 0.90 and not primary.get('drift_detected', False):
    print(f"   ✅ Model is production-ready")
    print(f"   • Establish baseline metrics")
    print(f"   • Set up continuous monitoring")
    print(f"   • Define retraining triggers")


📊 RESILIENCE TESTING SUMMARY

📈 OVERALL RESILIENCE:
   Score: 1.000

🎯 QUALITY ASSESSMENT:
   ✅ Resilience: EXCELLENT (score ≥ 0.90)
      Model is highly stable under distribution shifts

💡 RECOMMENDATIONS:
   ✅ Model is production-ready
   • Establish baseline metrics
   • Set up continuous monitoring
   • Define retraining triggers


## 9️⃣ Practical Decision Examples

How to use resilience metrics in real-world scenarios

In [9]:
print("\n💼 PRACTICAL USE CASES\n" + "="*70)

# Scenario 1: Medical Deployment
print(f"\n🏥 MEDICAL DEPLOYMENT - Hospital Transfer\n" + "-"*70)
resilience_score = 0.82
drift_detected = False
data_drift = 0.15

print(f"   Scenario: Deploying model from Hospital A to Hospital B")
print(f"   Resilience Score: {resilience_score:.2f}")
print(f"   Drift Detected: {drift_detected}")
print(f"   Data Drift: {data_drift:.2f}")

print(f"\n   📋 DECISION PROTOCOL:")
if resilience_score >= 0.80 and not drift_detected:
    print(f"   ✅ APPROVED for deployment")
    print(f"   Reason: High resilience, no significant drift")
    print(f"   Action: Deploy with standard monitoring")
    print(f"   Schedule: Weekly performance reviews")
elif resilience_score >= 0.70:
    print(f"   🟡 CONDITIONAL approval")
    print(f"   Reason: Acceptable resilience but requires caution")
    print(f"   Action: Deploy with enhanced monitoring")
    print(f"   Requirement: Daily performance checks for first month")
else:
    print(f"   ⚠️  REJECTED - Retrain required")
    print(f"   Reason: Insufficient resilience for critical application")
    print(f"   Action: Collect data from Hospital B and retrain")

# Scenario 2: Credit Scoring - Economic Change
print(f"\n\n💰 CREDIT SCORING - Economic Shift\n" + "-"*70)
resilience_score = 0.65
drift_detected = True
concept_drift = 0.35

print(f"   Scenario: Model trained pre-pandemic, evaluated post-pandemic")
print(f"   Resilience Score: {resilience_score:.2f}")
print(f"   Drift Detected: {drift_detected}")
print(f"   Concept Drift: {concept_drift:.2f}")

print(f"\n   📋 DECISION PROTOCOL:")
if drift_detected and concept_drift > 0.3:
    print(f"   🔴 IMMEDIATE ACTION REQUIRED")
    print(f"   Reason: High concept drift - target relationships changed")
    print(f"   Action: STOP using model for automated decisions")
    print(f"   Next Steps:")
    print(f"      1. Manual review for all applications")
    print(f"      2. Collect recent labeled data")
    print(f"      3. Retrain model with updated data")
    print(f"      4. Re-run resilience testing")

# Scenario 3: Fraud Detection - Evolving Tactics
print(f"\n\n🔒 FRAUD DETECTION - Evolving Threats\n" + "-"*70)
resilience_score = 0.88
drift_detected = True
data_drift = 0.22
concept_drift = 0.12

print(f"   Scenario: Quarterly resilience check on fraud detection model")
print(f"   Resilience Score: {resilience_score:.2f}")
print(f"   Drift Detected: {drift_detected}")
print(f"   Data Drift: {data_drift:.2f}")
print(f"   Concept Drift: {concept_drift:.2f}")

print(f"\n   📋 DECISION PROTOCOL:")
if resilience_score >= 0.85 and concept_drift < 0.2:
    print(f"   🟡 CONTINUE with ENHANCED MONITORING")
    print(f"   Reason: High resilience but moderate data drift")
    print(f"   Action: Keep model active with adjustments")
    print(f"   Adjustments:")
    print(f"      • Increase manual review threshold from 70% to 60%")
    print(f"      • Schedule retraining in 1 month")
    print(f"      • Analyze drifted features for new fraud patterns")
    print(f"      • Implement feature importance monitoring")


💼 PRACTICAL USE CASES

🏥 MEDICAL DEPLOYMENT - Hospital Transfer
----------------------------------------------------------------------
   Scenario: Deploying model from Hospital A to Hospital B
   Resilience Score: 0.82
   Drift Detected: False
   Data Drift: 0.15

   📋 DECISION PROTOCOL:
   ✅ APPROVED for deployment
   Reason: High resilience, no significant drift
   Action: Deploy with standard monitoring
   Schedule: Weekly performance reviews


💰 CREDIT SCORING - Economic Shift
----------------------------------------------------------------------
   Scenario: Model trained pre-pandemic, evaluated post-pandemic
   Resilience Score: 0.65
   Drift Detected: True
   Concept Drift: 0.35

   📋 DECISION PROTOCOL:
   🔴 IMMEDIATE ACTION REQUIRED
   Reason: High concept drift - target relationships changed
   Action: STOP using model for automated decisions
   Next Steps:
      1. Manual review for all applications
      2. Collect recent labeled data
      3. Retrain model with updated da

## 🔟 Visualize Model Stability

Visualize how model performance varies across different data subsets

In [10]:
# Simulate performance on different data distributions
n_simulations = 20
performance_scores = []
drift_levels = []

np.random.seed(42)

for i in range(n_simulations):
    # Simulate varying drift levels
    drift_level = np.random.uniform(0, 0.5)
    drift_levels.append(drift_level)
    
    # Performance degrades with drift
    base_performance = 0.95
    performance = base_performance - (drift_level * 0.4) + np.random.normal(0, 0.02)
    performance_scores.append(max(0.5, min(1.0, performance)))

# Create visualization
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Plot 1: Performance vs Drift
scatter = ax1.scatter(drift_levels, performance_scores, 
                     c=performance_scores, s=100, 
                     cmap='RdYlGn', alpha=0.6, edgecolors='black')
ax1.plot(drift_levels, performance_scores, 'b--', alpha=0.3, label='Trend')
ax1.axhline(y=0.90, color='green', linestyle='--', linewidth=2, label='Good Performance')
ax1.axhline(y=0.75, color='orange', linestyle='--', linewidth=2, label='Acceptable')
ax1.axhline(y=0.60, color='red', linestyle='--', linewidth=2, label='Poor')
ax1.set_xlabel('Drift Level', fontsize=12, fontweight='bold')
ax1.set_ylabel('Model Performance', fontsize=12, fontweight='bold')
ax1.set_title('Performance Degradation vs Data Drift', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0.5, 1.05)
plt.colorbar(scatter, ax=ax1, label='Performance')

# Plot 2: Performance Distribution
ax2.hist(performance_scores, bins=15, color='skyblue', alpha=0.7, edgecolor='black')
ax2.axvline(x=np.mean(performance_scores), color='red', linestyle='--', 
           linewidth=2, label=f'Mean: {np.mean(performance_scores):.3f}')
ax2.axvline(x=np.median(performance_scores), color='green', linestyle='--', 
           linewidth=2, label=f'Median: {np.median(performance_scores):.3f}')
ax2.set_xlabel('Performance Score', fontsize=12, fontweight='bold')
ax2.set_ylabel('Frequency', fontsize=12, fontweight='bold')
ax2.set_title('Distribution of Performance Scores', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n📊 Stability Statistics:")
print(f"   Mean Performance: {np.mean(performance_scores):.3f}")
print(f"   Std Performance: {np.std(performance_scores):.3f}")
print(f"   Min Performance: {np.min(performance_scores):.3f}")
print(f"   Max Performance: {np.max(performance_scores):.3f}")
print(f"   Performance Range: {np.max(performance_scores) - np.min(performance_scores):.3f}")

stability_score = 1 - np.std(performance_scores)
print(f"\n   Stability Score: {stability_score:.3f}")
if stability_score >= 0.95:
    print(f"   ✅ Highly stable model")
elif stability_score >= 0.90:
    print(f"   🟡 Moderately stable model")
else:
    print(f"   ⚠️  Model shows significant variability")


📊 Stability Statistics:
   Mean Performance: 0.877
   Std Performance: 0.048
   Min Performance: 0.793
   Max Performance: 0.949
   Performance Range: 0.157

   Stability Score: 0.952
   ✅ Highly stable model


/tmp/ipykernel_1999457/2253643128.py:50: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## 1️⃣1️⃣ Generate Static Report (Optional)

Generate a static HTML report with matplotlib charts (for PDF export)

In [11]:
# Generate static HTML report
static_html_path = os.path.join(output_dir, 'resilience_classification_static.html')

print("📝 Generating static HTML report...\n")

static_report_path = exp.save_html(
    test_type='resilience',
    file_path=static_html_path,
    model_name='RandomForest Classifier',
    report_type='static'  # Uses matplotlib instead of Plotly
)

print(f"\n✅ Static report generated!")
print(f"📂 Location: {static_report_path}")
print(f"\n💡 Static reports can be easily printed or converted to PDF")

📝 Generating static HTML report...

2025-11-12 22:40:27,340 - deepbridge.reports - INFO - Using static renderer for resilience report
2025-11-12 22:40:27,341 - deepbridge.reports - INFO - Generating static resilience report to: /home/guhaase/projetos/DeepBridge/examples/notebooks/03_validation_tests/outputs/resilience_classification/resilience_classification_static.html
2025-11-12 22:40:27,343 - deepbridge.reports - INFO - Found template at: /home/guhaase/projetos/DeepBridge/deepbridge/templates/report_types/resilience/static/index.html
2025-11-12 22:40:27,343 - deepbridge.reports - INFO - Found resilience template: /home/guhaase/projetos/DeepBridge/deepbridge/templates/report_types/resilience/static/index.html
2025-11-12 22:40:27,344 - deepbridge.reports - INFO - Using static template: /home/guhaase/projetos/DeepBridge/deepbridge/templates/report_types/resilience/static/index.html
2025-11-12 22:40:27,346 - deepbridge.reports - INFO - CSS compiled successfully using CSSManager for stat

## 🎉 Summary - Files Generated

### 📂 Output Directory Structure:
```
outputs/resilience_classification/
├── resilience_classification_interactive.html  # Interactive report with Plotly
├── resilience_classification_static.html       # Static report with Matplotlib
├── resilience_classification_results.json      # Complete results in JSON
└── resilience_classification_results_COMPACT.json  # Compact JSON for AI
```

### ✅ What You Learned:

1. **Resilience Testing**
   - Detect data drift (covariate shift)
   - Detect concept drift (relationship changes)
   - Assess model stability under distribution shifts
   - Calculate resilience scores

2. **Report Generation**
   - Interactive HTML reports with Plotly
   - Static HTML reports with Matplotlib
   - Complete control over report type

3. **JSON Export**
   - Full experiment metadata
   - Drift detection metrics
   - Per-feature drift analysis
   - Resilience scores and recommendations
   - Easy integration with monitoring systems

4. **Practical Applications**
   - Hospital deployment decision protocols
   - Credit scoring with economic shifts
   - Fraud detection with evolving threats
   - Production monitoring strategies

### 💡 Best Practices:

- ✅ Run resilience tests before production deployment
- ✅ Establish baseline metrics in development
- ✅ Set up continuous drift monitoring
- ✅ Define clear retraining triggers
- ✅ Monitor both data drift and concept drift
- ✅ Export JSON for automated monitoring pipelines

### 🚀 Next Steps:

- 📘 `02_complete_robustness.ipynb` - Model robustness testing
- 📘 `03_uncertainty_classification.ipynb` - Uncertainty quantification
- 📘 `../04_fairness/` - Fairness and bias analysis

<div style="background-color: #e8f5e9; padding: 15px; border-radius: 5px; border-left: 5px solid #4caf50;">
<b>🎯 Key Takeaway:</b> Resilience testing is essential for maintaining model reliability in production. Regular monitoring and retraining based on drift metrics prevent silent model degradation and ensure continued accuracy.
</div>